In [1]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import pandas as pd
import numpy as np

# Atributos que serão extraídos

1. id
2. nome do produto
3. tipo do produto
4. cor do produto 
5. composição do produto
6. preço

# Web Scrapping

In [2]:
#url em que a requisição será feita
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

#fazendo a requisição
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
page = requests.get(url, headers=headers)

#pegando o html da página
html = page.text

In [3]:
#criando um objeto a partir do html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
#fazendo a requisição
page = requests.get(url, headers)

In [5]:
#pegando o html da página
html = page.text

### Coletando id, nome, tipo e preço

In [6]:
#buscando pela lista que contém todos produtos
all_products = soup.find('ul', class_='products-listing small')

#fazendo uma lista que tem o html de cada produto
products_list = all_products.find_all('article', class_='hm-product-item')

#lista para armazenar tipo e preço de cada produto
products_id, products_type, products_price, products_name = [],[],[],[]

#iterando em todos os produtos e pegando preço e tipo
for product in products_list:
    product_details = product.find('div', class_='item-details')
    products_id.append(product.get('data-articlecode'))
    products_type.append(product.get('data-category'))
    products_price.append(product_details.find('span', class_='price regular').get_text())
    products_name.append(product_details.find('a', class_='link').get_text())

#criando um dataframe com os dados coletando até agora

df = pd.DataFrame([products_id, products_name, products_type, products_price]).T
df.columns = ['product_id', 'product_name', 'product_type', 'product_price']
df['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [7]:
df.head()

,product_id,product_name,product_type,product_price,scrapy_datetime
0,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-12-15 20:56:52
1,0690449067,Skinny Jeans,men_jeans_ripped,$ 39.99,2022-12-15 20:56:52
2,0985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-12-15 20:56:52
3,1008549001,Regular Jeans,men_jeans_regular,$ 24.99,2022-12-15 20:56:52
4,1008549006,Regular Jeans,men_jeans_regular,$ 24.99,2022-12-15 20:56:52


### Coletando os atributos id, nome, tipo e preço de todos os produtos

No exemplo acima, coletamos apenas o 36 primeiros items da vitrine. Aqueles que aparecem na primeira página. Nessa segunda versão, estamos passando por todas as páginas daquele produto e coletando todos os produtos.

In [8]:
#coletando quantos items existem ao total daquela categoria específica
total_items = int(soup.find('h2', class_='load-more-heading').get('data-total'))

#considerando que cada página mostra 36 produtos, vemos quantas paginas precisam
page_number = int(round(total_items/36))

#url para pegar todos os produtos
url2 = url + "?page-size=" + str(page_number*36)

#fazendo a requisição
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
page = requests.get(url2, headers=headers)

#pegando o html da página
html = page.text

#criando um objeto do BeautifulSoup a partir do html
soup_pagination = BeautifulSoup(html)

#buscando pela lista que contém todos produtos
all_products = soup_pagination.find('ul', class_='products-listing small')

#fazendo uma lista que tem o html de cada produto
products_list = all_products.find_all('article', class_='hm-product-item')

#lista para armazenar tipo e preço de cada produto
products_id, products_type, products_price, products_name = [],[],[],[]

#iterando em todos os produtos e pegando preço e tipo
for product in products_list:
    product_details = product.find('div', class_='item-details')
    products_id.append(product.get('data-articlecode'))
    products_type.append(product.get('data-category'))
    products_price.append(product_details.find('span', class_='price regular').get_text())
    products_name.append(product_details.find('a', class_='link').get_text())

#criando um dataframe com os dados coletando até agora
df = pd.DataFrame([products_id, products_name, products_type, products_price]).T
df.columns = ['product_id', 'product_name', 'product_type', 'product_price']
df['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [9]:
df.shape

(91, 5)

In [10]:
df.head()

,product_id,product_name,product_type,product_price,scrapy_datetime
0,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-12-15 20:56:55
1,0985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-12-15 20:56:55
2,0690449067,Skinny Jeans,men_jeans_ripped,$ 39.99,2022-12-15 20:56:55
3,0690449036,Skinny Jeans,men_jeans_ripped,$ 39.99,2022-12-15 20:56:55
4,0690449022,Skinny Jeans,men_jeans_ripped,$ 39.99,2022-12-15 20:56:55


### Coletando as cores e a composição de cada produto

In [11]:
def get_colors_and_composition(id):
    #url em que a requisição será feita
    url_model = f'https://www2.hm.com/en_us/productpage.{id}.html'

    #fazendo a requisição
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
    page = requests.get(url_model, headers=headers)

    #pegando o html da página
    html = page.text

    #criando um objeto do BeautifulSoup a partir do html
    soup_product = BeautifulSoup(html)
    
    # ====================colors===========================
    
    #pegando todos os tipos de cores
    product_colors = soup_product.find('ul', class_='inputlist clearfix')

    #cores disponíveis daquele produto
    colors = [product_item.find('a').get('data-color') for product_item in product_colors.find_all('li', class_='list-item')]
    
    # ====================composition===========================
    attributes = list(filter(None, soup_product.find('div', class_='content pdp-text pdp-content').get_text().split('\n')))

    for i in range(len(attributes)):
        if attributes[i] == 'Composition':
                composition_shell = attributes[i+1]
                composition_pocket = attributes[i+2]
    
    
    return "//".join(colors), composition_shell, composition_pocket

In [12]:
#lista para armazenar as cores 
products_colors = []

#lista para armazenar a composição
products_composition_shell = []
products_composition_pocket = []

#para cada produto pegar as cores e adicionar na lista
for id in df['product_id']:
    colors, composition_shell, composition_pocket = get_colors_and_composition(id)
    products_colors.append(colors)
    products_composition_shell.append(composition_shell)
    products_composition_pocket.append(composition_pocket)    

#adiciona as colunas de cores e composição do produto no dataframe
df["product_colors"] = products_colors
df["product_composition_shell"] = products_composition_shell
df["product_composition_pocket"] = products_composition_pocket

#reordenando as colunas
df = df[["product_id", "product_name", "product_type", "product_colors", "product_price", "product_composition_shell", "product_composition_pocket", "scrapy_datetime"]]

#se a string não começar com "Poc" significa que esse valor não estava presente no site
df["product_composition_pocket"] = df["product_composition_pocket"].apply(lambda x: x if x[:3] == "Poc" else np.nan)

#limpando as colunas de composição do produto
df["product_composition_shell"] = df['product_composition_shell'].str.replace('Shell: ', '')
df["product_composition_pocket"] = df['product_composition_pocket'].str.replace('Pocket lining: ', '')
df["product_composition_pocket"] = df['product_composition_pocket'].str.replace('Pocket: ', '')

In [13]:
df.head()

,product_id,product_name,product_type,product_colors,product_price,product_composition_shell,product_composition_pocket,scrapy_datetime
0,1024256001,Slim Jeans,men_jeans_slim,Black//Light denim blue//Light denim blue//Den...,$ 19.99,"Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",2022-12-15 20:56:55
1,0985159001,Skinny Jeans,men_jeans_skinny,Black//Denim blue//Dark gray//Light denim blue...,$ 24.99,"Cotton 99%, Spandex 1%",Cotton 100%,2022-12-15 20:56:55
2,0690449067,Skinny Jeans,men_jeans_ripped,Light denim blue/trashed//Denim blue//Black/wa...,$ 39.99,"Cotton 98%, Spandex 2%","Polyester 65%, Cotton 35%",2022-12-15 20:56:55
3,0690449036,Skinny Jeans,men_jeans_ripped,Light denim blue/trashed//Denim blue//Black/wa...,$ 39.99,"Cotton 98%, Spandex 2%",NaN,2022-12-15 20:56:55
4,0690449022,Skinny Jeans,men_jeans_ripped,Light denim blue/trashed//Denim blue//Black/wa...,$ 39.99,"Cotton 98%, Spandex 2%",NaN,2022-12-15 20:56:55


### Uma linha por produto-cor

In [14]:
#dataframe final
df_raw = pd.DataFrame()

In [15]:
#para cada produto do df, extrai as cores e cria uma linha pra cada cor
for i in range(len(df)):
    #pega a linha
    df_row = pd.DataFrame(df.loc[i]).T
    #pega as cores daquele produto
    unique_colors = df_row["product_colors"].values[0].split("//")
    
    #pra cada cor, cria uma nova linha
    for i in range(len(unique_colors)):
        df_row["product_colors"] = unique_colors[i]
        df_raw = pd.concat([df_raw, df_row])

In [16]:
#exportando o dataframe gerado para um arquivo csv
df_raw.to_csv("./generated_data/df_raw_scrapping.csv")